In [229]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from textblob import TextBlob
from datetime import date, datetime
from pytz import timezone

In [230]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [231]:
df = pd.read_sql_query("SELECT * FROM spark_cleaned", con)
df = df.rename(columns = {'date_time': 'date_time_est'})

In [222]:
print(df.iloc[0, 2])
type(df.iloc[0,2])

2021-09-04 12:48:31.000000


str

In [237]:
eastern = timezone('US/Eastern')
utc = timezone('UTC')

# "%d/%m/%Y %H:%M:%S"
for index, row in df['date_time_est'].iteritems():
    
    created_at = datetime.strptime(row[:-7], '%Y-%m-%d %H:%M:%S')
    utc_created_at = utc.localize(created_at)
    #print(utc_created_at)
    row = utc_created_at.astimezone(eastern)
    df.loc[index, 'date_time_est'] = row
    #print(est_created_at)
    #print(type(row))

In [238]:
df.head(5)

,index,screen_name,date_time_est,text,retweeted,lang
0,0,Zed18978371,2021-09-04 08:48:31-04:00,@NorristownN @RolandStautz Yup - I’m the crazy...,0.0,en
1,1,TeslaOwls,2021-09-04 09:28:16-04:00,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0.0,en
2,2,vin_wiesel,2021-09-04 09:28:11-04:00,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0.0,en
3,3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,@TeslaAIBot @Tesla Why you have so many follow...,0.0,en
4,4,developrwannab,2021-09-04 09:28:53-04:00,@ukspacebulletin Great thanks. Now I got to ad...,0.0,en


In [239]:
df_drop_index = df.drop(columns = 'index')

In [240]:
df_drop_index.head()

,screen_name,date_time_est,text,retweeted,lang
0,Zed18978371,2021-09-04 08:48:31-04:00,@NorristownN @RolandStautz Yup - I’m the crazy...,0.0,en
1,TeslaOwls,2021-09-04 09:28:16-04:00,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0.0,en
2,vin_wiesel,2021-09-04 09:28:11-04:00,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0.0,en
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,@TeslaAIBot @Tesla Why you have so many follow...,0.0,en
4,developrwannab,2021-09-04 09:28:53-04:00,@ukspacebulletin Great thanks. Now I got to ad...,0.0,en


In [241]:
len(df_drop_index)

19308

In [242]:
# remove duplicates 
df_no_dupes = df_drop_index.drop_duplicates()
df_no_dupes
len(df_no_dupes)

17740

In [243]:
# use regex to remove @usernames

df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
#re.sub('@[^\s]+','',Tweet)

<ipython-input-243-59960d4ecb67>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
<ipython-input-243-59960d4ecb67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')


In [244]:
df_no_dupes

,screen_name,date_time_est,text,retweeted,lang
0,Zed18978371,2021-09-04 08:48:31-04:00,Yup - I’m the crazy crypto/Tesla guy..,0.0,en
1,TeslaOwls,2021-09-04 09:28:16-04:00,I don't believe I ever claimed Tesla had ...,0.0,en
2,vin_wiesel,2021-09-04 09:28:11-04:00,Media: “TESLA CRASHES INTO BUILDING WHILE ON ...,0.0,en
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,Why you have so many followers ?,0.0,en
4,developrwannab,2021-09-04 09:28:53-04:00,Great thanks. Now I got to add Tesla bot 🤔 So...,0.0,en
...,...,...,...,...,...
19303,NobleChristos,2021-09-04 23:39:32-04:00,UNIQUE NFT SIGNED BY \n\nBid below to win this...,NaN,en
19304,manyfacedgoddd,2021-09-04 23:39:19-04:00,Tesla going to $0,NaN,en
19305,bruceba12492082,2021-09-04 23:39:01-04:00,https://t.co/KRZbDNGqRk &lt;---- THE MOST TRUS...,NaN,en
19306,VtopiaModerna,2021-09-04 23:38:46-04:00,Tesla of the skies: This French start-up wants...,NaN,en


In [245]:
# use regex to remove punctuation and emojis 
df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')

<ipython-input-245-0fde4d1c2c14>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')
<ipython-input-245-0fde4d1c2c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')


In [246]:
df_no_dupes

,screen_name,date_time_est,text,retweeted,lang
0,Zed18978371,2021-09-04 08:48:31-04:00,Yup Im the crazy cryptoTesla guy,0.0,en
1,TeslaOwls,2021-09-04 09:28:16-04:00,I dont believe I ever claimed Tesla had g...,0.0,en
2,vin_wiesel,2021-09-04 09:28:11-04:00,Media TESLA CRASHES INTO BUILDING WHILE ON AU...,0.0,en
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,Why you have so many followers,0.0,en
4,developrwannab,2021-09-04 09:28:53-04:00,Great thanks Now I got to add Tesla bot So t...,0.0,en
...,...,...,...,...,...
19303,NobleChristos,2021-09-04 23:39:32-04:00,UNIQUE NFT SIGNED BY \n\nBid below to win this...,NaN,en
19304,manyfacedgoddd,2021-09-04 23:39:19-04:00,Tesla going to 0,NaN,en
19305,bruceba12492082,2021-09-04 23:39:01-04:00,httpstcoKRZbDNGqRk lt THE MOST TRUSTED CHANNEL...,NaN,en
19306,VtopiaModerna,2021-09-04 23:38:46-04:00,Tesla of the skies This French startup wants t...,NaN,en


In [247]:
# select only certain time periods, keeping note gmt to est 


In [248]:
# make all text lowercase

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())

<ipython-input-248-4aad2dac7cc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())


In [249]:
# remove stop words 
stop = stopwords.words('english')

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

<ipython-input-249-e64ccaa07b0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [250]:
# stop
# list of stop words

In [251]:
# clean up indexes
df = df_no_dupes
df = df.reset_index(drop=True)
df

,screen_name,date_time_est,text,retweeted,lang
0,Zed18978371,2021-09-04 08:48:31-04:00,yup im crazy cryptotesla guy,0.0,en
1,TeslaOwls,2021-09-04 09:28:16-04:00,dont believe ever claimed tesla great build qu...,0.0,en
2,vin_wiesel,2021-09-04 09:28:11-04:00,media tesla crashes building autopilot,0.0,en
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,many followers,0.0,en
4,developrwannab,2021-09-04 09:28:53-04:00,great thanks got add tesla bot thats kind deve...,0.0,en
...,...,...,...,...,...
17735,NobleChristos,2021-09-04 23:39:32-04:00,unique nft signed bid win genuine signed tesla...,NaN,en
17736,manyfacedgoddd,2021-09-04 23:39:19-04:00,tesla going 0,NaN,en
17737,bruceba12492082,2021-09-04 23:39:01-04:00,httpstcokrzbdngqrk lt trusted channel crypto s...,NaN,en
17738,VtopiaModerna,2021-09-04 23:38:46-04:00,tesla skies french startup wants take zeroemis...,NaN,en


In [252]:
# sentiment analysis packages

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [253]:
# nlp 

df[['polarity', 'subjectivity']] = df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [254]:
df.head()

,screen_name,date_time_est,text,retweeted,lang,polarity,subjectivity
0,Zed18978371,2021-09-04 08:48:31-04:00,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000
1,TeslaOwls,2021-09-04 09:28:16-04:00,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909
2,vin_wiesel,2021-09-04 09:28:11-04:00,media tesla crashes building autopilot,0.0,en,0.000000,0.000000
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,many followers,0.0,en,0.500000,0.500000
4,developrwannab,2021-09-04 09:28:53-04:00,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500


In [255]:
# NLP for sentiment, neg, neu, pos, compound

for index, row in df['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    #print(type(score))
    #print(score)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        # loc looks at curent row and if neg > pos, sentiment is negative
        df.loc[index, 'sentiment'] = 'negative'
    elif pos > neg:
        df.loc[index, 'sentiment'] = 'positive'
    else:
        df.loc[index, 'sentiment'] = 'neutral'
        
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp
    

In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17740 entries, 0 to 17739
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   screen_name    17740 non-null  object 
 1   date_time_est  17740 non-null  object 
 2   text           17740 non-null  object 
 3   retweeted      81 non-null     float64
 4   lang           17740 non-null  object 
 5   polarity       17740 non-null  float64
 6   subjectivity   17740 non-null  float64
 7   sentiment      17740 non-null  object 
 8   neg            17740 non-null  float64
 9   neu            17740 non-null  float64
 10  pos            17740 non-null  float64
 11  compound       17740 non-null  float64
dtypes: float64(7), object(5)
memory usage: 1.6+ MB


In [258]:
# data lake > spark cleaned > NLP cleaned > visualization 
df.head(10)

,screen_name,date_time_est,text,retweeted,lang,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,Zed18978371,2021-09-04 08:48:31-04:00,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000,negative,0.375,0.625,0.000,-0.3400
1,TeslaOwls,2021-09-04 09:28:16-04:00,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909,positive,0.175,0.585,0.240,0.2960
2,vin_wiesel,2021-09-04 09:28:11-04:00,media tesla crashes building autopilot,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
3,Jeeeeee66023780,2021-09-04 09:28:09-04:00,many followers,0.0,en,0.500000,0.500000,neutral,0.000,1.000,0.000,0.0000
4,developrwannab,2021-09-04 09:28:53-04:00,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500,positive,0.000,0.423,0.577,0.9468
5,luke_deluxe,2021-09-04 09:28:44-04:00,fine tesla,0.0,en,0.416667,0.500000,positive,0.000,0.357,0.643,0.2023
6,lordhyperkill,2021-09-04 10:52:30-04:00,thats real thing whenever heard thought people...,0.0,en,-0.066667,0.233333,positive,0.000,0.929,0.071,0.2263
7,barryagall,2021-09-04 10:52:29-04:00,white interior,0.0,en,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
8,future_yas,2021-09-05 16:48:38-04:00,uncle rich made tesla bot hilarious episode bu...,0.0,en,0.437500,0.875000,positive,0.000,0.717,0.283,0.7430
9,thatdavemarsh,2021-09-05 16:48:35-04:00,angle tesla doesnt repair procedures extent ty...,0.0,en,0.500000,0.500000,negative,0.168,0.681,0.152,-0.0772
